In [1]:
import os
os.chdir("/root/workspace/sjh/data/paper")
from os.path import join as pj
import argparse
import sys
sys.path.append("modules")
import utils
import numpy as np
import scib
import scib.metrics as me
import anndata as ad
import scipy
import pandas as pd
import re
from glob import glob
import copy

R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: MatrixGenerics

R[write to console]: Loading required package: matrixStats

R[write to console]: 
Attaching package: ‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowC

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--task', type=str, default='dm_sub10')
# parser.add_argument('--ref_task', type=str, default='bmmc_sub100')
parser.add_argument('--experiment', type=str, default='mask_0.3')
parser.add_argument('--model', type=str, default='default')
parser.add_argument('--method', type=str, default='embed')
o, _ = parser.parse_known_args()  # for python interactive
test_dir = "predict"
# o = parser.parse_args()
start = 999
end = 1000
step = 100

In [3]:
cfg_task = re.sub("_vd.*|_vt.*|_atlas|_generalize|_transfer|_ref_.*", "", o.task)
data_config = utils.load_toml("configs/data.toml")[cfg_task]
for k, v in data_config.items():
    vars(o)[k] = v
model_config = utils.load_toml("configs/model.toml")["default"]
if o.model != "default":
    model_config.update(utils.load_toml("configs/model.toml")[o.model])
for k, v in model_config.items():
    vars(o)[k] = v
o.s_joint, o.combs, *_ = utils.gen_all_batch_ids(o.s_joint, o.combs)

In [4]:
# Load cell type labels
labels = []
batch = []
labels_dir = []
for raw_data_dir in o.raw_data_dirs:
    for s in o.s_joint:
        labels_dir += glob(pj("/dev/shm/processed/", o.task, "subset_" + str(s), "labels.csv"))
        # labels_dir += glob(pj("/dev/shm/processed/", o.ref_task, "subset_" + str(s), "labels.csv"))

p = 0
for l in labels_dir:
    label = utils.load_csv(l)[1:]
    labels += utils.transpose_list(label)[6]
    batch += [str(p)] * len(label)
    p += 1

labels = np.array(labels)
batch = np.array(batch)

print(list(set(labels)))
print(list(set(batch)))

['Pericytes', 'Fibroblasts', 'Oligodendrocytes', 'Epithelial cells', 'Endothelial cells', 'Proliferating Erythrocytes', 'Erythrocytes', 'Ependymal cells']
['0']


In [5]:
def bio_evaluate(o, pred, labels, result_dir):
    if o.method in ["embed", "mofa", "scmomat", "stabmap", "scvaeit", "multigrate", "glue"]:
        output_type = "embed"
    elif o.method in [
        "midas_feat+wnn", 
        "harmony+wnn", 
        "pca+wnn",
        "seurat_cca+wnn",
        "seurat_rpca+wnn",
        "scanorama_embed+wnn",
        "scanorama_feat+wnn",
        "liger+wnn",
        "bbknn",
        ]:
        output_type = "graph"
    else:
        assert False, o.method+": invalid method!"

    embed = "X_emb"
    batch_key = "batch"
    label_key = "label"
    cluster_key = "cluster"
    si_metric = "euclidean"
    subsample = 0.5
    verbose = False

    c = pred["z"]["joint"][:, :o.dim_c]
    s = pred["s"]["joint"]

    if o.method == "embed":
        adata = ad.AnnData(c)
        adata.obsm[embed] = c
        adata.obs[batch_key] = s.astype(str)
        adata.obs[batch_key] = adata.obs[batch_key].astype("category")
        adata.obs[label_key] = labels
        adata.obs[label_key] = adata.obs[label_key].astype("category")
    elif o.method in ["mofa", "stabmap", "multigrate", "glue"]:
        adata = ad.AnnData(c*0)
        embeddings = utils.load_csv(pj(result_dir, "embeddings.csv"))
        adata.obsm[embed] = np.array(embeddings)[1:, 1:].astype(np.float32)
        adata.obs[batch_key] = s.astype(str)
        adata.obs[batch_key] = adata.obs[batch_key].astype("category")
        adata.obs[label_key] = labels
        adata.obs[label_key] = adata.obs[label_key].astype("category")
    elif o.method in ["scmomat", "scvaeit"]:
        adata = ad.AnnData(c*0)
        embeddings = utils.load_csv(pj(result_dir, "embeddings.csv"))
        adata.obsm[embed] = np.array(embeddings).astype(np.float32)
        adata.obs[batch_key] = s.astype(str)
        adata.obs[batch_key] = adata.obs[batch_key].astype("category")
        adata.obs[label_key] = labels
        adata.obs[label_key] = adata.obs[label_key].astype("category")
    elif o.method in [
        "midas_feat+wnn", 
        "harmony+wnn", 
        "pca+wnn",
        "seurat_cca+wnn",
        "seurat_rpca+wnn",
        "scanorama_embed+wnn",
        "scanorama_feat+wnn",
        "liger+wnn",
        "bbknn",
        ]:
        adata = ad.AnnData(c*0)
        adata.obs[batch_key] = s.astype(str)
        adata.obs[batch_key] = adata.obs[batch_key].astype("category")
        adata.obs[label_key] = labels
        adata.obs[label_key] = adata.obs[label_key].astype("category")
        adata.obsp["connectivities"] = scipy.io.mmread(pj(result_dir, "connectivities.mtx")).tocsr()
        adata.uns["neighbors"] = {'connectivities_key': 'connectivities'}

    results = {}
    # print(adata)

    print('clustering...')
    res_max, nmi_max, nmi_all = scib.clustering.opt_louvain(adata, label_key=label_key,
        cluster_key=cluster_key, function=me.nmi, use_rep=embed, verbose=verbose, inplace=True)
    
    # results["asw_batch"] = me.silhouette_batch(adata, batch_key=batch_key,
    #         group_key=label_key, embed=embed, metric=si_metric, verbose=verbose)
    
    results['NMI'] = me.nmi(adata, group1=cluster_key, group2=label_key, method='arithmetic')
    print("NMI: " + str(results['NMI']))

    results['ARI'] = me.ari(adata, group1=cluster_key, group2=label_key)
    print("ARI: " + str(results['ARI']))

    # type_ = "knn" if output_type == "graph" else None
    # results['kBET'] = me.kBET(adata, batch_key=batch_key, label_key=label_key, embed=embed, 
    #     type_=type_, verbose=verbose, return_df = False)
    # print("kBET: " + str(results['kBET']))

    results['il_score_asw'] = me.isolated_labels(adata, label_key=label_key, batch_key=batch_key,
        embed=embed, cluster=False, verbose=verbose)
    print("il_score_asw: " + str(results['il_score_asw']))

    # results['graph_conn'] = me.graph_connectivity(adata, label_key=label_key)
    # print("graph_conn: " + str(results['graph_conn']))

    # results['cLISI'] = me.clisi_graph(adata, batch_key=batch_key, label_key=label_key, type_="knn",
    #     subsample=subsample*100, n_cores=1, verbose=verbose)
    # print("cLISI: " + str(results['cLISI']))

    # results['iLISI'] = me.ilisi_graph(adata, batch_key=batch_key, type_="knn",
    #     subsample=subsample*100, n_cores=1, verbose=verbose)
    # print("iLISI: " + str(results['iLISI']))

    results = {k: float(v) for k, v in results.items()}
    # results['batch_score'] = np.nanmean([results['iLISI'], results['graph_conn'], results['kBET']])
    results['bio_score'] = np.nanmean([results['NMI'], results['ARI'], results['il_score_asw']])
    # results["overall_score"] = float(0.4 * results['batch_score'] + 0.6 * results['bio_score'])

    df = pd.DataFrame({
        # 'iLISI':          [results['iLISI']],
        # 'graph_conn':     [results['graph_conn']],
        # 'kBET':           [results['kBET']],
        # # 'asw_batch':      [results['asw_batch']],
        # 'batch_score':    [results['batch_score']],
        'NMI':            [results['NMI']],
        'ARI':            [results['ARI']],
        'il_score_asw':    [results['il_score_asw']],
        # 'cLISI':          [results['cLISI']],
        'bio_score':      [results['bio_score']],
        # 'overall_score':  [results['overall_score']]
    })
    print(df)

    return df

In [6]:
o.mods = ["rna", "adt"]
df_batch_bio_embed = []

for i in list(range(start, end, step)):
    init_model = "sp_0000{:04d}".format(i)
    result_dir = pj("result", "comparison", o.task, o.experiment)
    utils.mkdirs(result_dir, remove_old = False)
    # Load predicted latent variables
    o.pred_dir = pj("result", o.task, o.experiment, o.model, test_dir, init_model)
    pred = utils.load_predicted(o)
    df_bio = bio_evaluate(o, pred, labels, result_dir)
    df_bio["Model"] = "Step_" + "{:04d}".format(i + 1)
    df_batch_bio_embed.append(df_bio)

Loading predicted variables ...
Converting to numpy ...
clustering...


/root/anaconda3/envs/private/lib/python3.8/site-packages/umap/distances.py:1051: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/root/anaconda3/envs/private/lib/python3.8/site-packages/umap/distances.py:1059: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/root/anaconda3/envs/private/lib/python3.8/site-packages/umap/distances.py:1074: Num

NMI: 0.7452647768780244
ARI: 0.8213679684478459
il_score_asw: 0.6255591651424766
        NMI       ARI  il_score_asw  bio_score
0  0.745265  0.821368      0.625559   0.730731


In [7]:
# # SLN
# df_bio_cat = pd.concat(df_batch_bio_embed, axis = 0)
# df_bio_mean_cat = copy.deepcopy(df_bio_cat)
# # df_bio_mean_cat["batch_score"] = df_bio_cat[["iLISI", "graph_conn", "kBET", "asw_batch"]].mean(axis = 1)
# df_bio_mean_cat["batch_score"] = df_bio_cat[["iLISI", "graph_conn", "kBET"]].mean(axis = 1)
# df_bio_mean_cat["bio_score"] = df_bio_cat[["NMI", "ARI", "il_score_asw", "cLISI"]].mean(axis = 1)
# df_bio_mean_cat = df_bio_mean_cat[["Model", "NMI", "ARI", "il_score_asw", "cLISI", "bio_score", "iLISI", "graph_conn", "kBET", "batch_score"]]
# # df_bio_mean_cat = df_bio_mean_cat[["Model", "NMI", "ARI", "il_score_asw", "cLISI", "bio_score", "iLISI", "graph_conn", "kBET", "asw_batch", "batch_score"]]
# # df_bio_mean_cat_sorted = df_bio_mean_cat.sort_values("bio_score", ascending = False, inplace = False)
# df_bio_mean_cat.to_csv(pj(result_dir, "metrics_bio_train.csv"), index = False)
# df_bio_mean_cat

In [8]:
# DM
df_bio_cat = pd.concat(df_batch_bio_embed, axis = 0)
df_bio_mean_cat = copy.deepcopy(df_bio_cat)
df_bio_mean_cat["bio_score"] = df_bio_cat[["NMI", "ARI", "il_score_asw"]].mean(axis = 1)
df_bio_mean_cat = df_bio_mean_cat[["Model", "NMI", "ARI", "il_score_asw", "bio_score"]]
# df_bio_mean_cat_sorted = df_bio_mean_cat.sort_values("bio_score", ascending = False, inplace = False)
# df_bio_mean_cat.to_csv(pj(result_dir, "metrics_bio_train.csv"), index = False)
df_bio_mean_cat

,Model,NMI,ARI,il_score_asw,bio_score
0,Step_1000,0.745265,0.821368,0.625559,0.730731
